# Bias model bootstrapping notebook

Eventually this notebook will (hopefully?) contain all the code necessary to run our final experiments.
For now it just contains a demo of MCE IRL on some gridworlds from the "learning biases" paper.

First we have some not-very-interesting setup:

In [ ]:
%matplotlib inline

# These are useful for debugging, but make code slower:
%load_ext autoreload
%autoreload 2

import logging

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import jax
import jax.experimental.optimizers as jopt
import jax.random as jrandom
import numpy as np

from pref_bootstrap.envs import gridworld, mdp_interface
from pref_bootstrap.algos import mce_irl
import pref_bootstrap.feedback_learner_blind_irl as fbl_blind_irl
import pref_bootstrap.feedback_learner_paired_comparisons as fbl_paired_comp
import pref_bootstrap.reward_models as r_models
import pref_bootstrap.expert_base as experts

sns.set(context='notebook', style='darkgrid')
logging.basicConfig(level=logging.INFO)
np.set_printoptions(precision=4, linewidth=100)

## Very simple example of using environments and MCE IRL code

This code doesn't use the new agent API

In [ ]:
random_gridworld = gridworld.GridworldMdp.generate_random(4, 4, 0.2, 0.1)
env = mdp_interface.GridworldEnvWrapper(random_gridworld, random_gridworld.height + random_gridworld.width)

In [ ]:
# generate some optimal demos (represented by a single optimal occupancy measure vector) and run MCE IRL
_, optimal_om = mce_irl.mce_occupancy_measures(env)
rmodel = r_models.LinearRewardModel(env.obs_dim)
opt_tup = jopt.momentum(1e-2, 0.9)

In [ ]:
rew_params, visitations = mce_irl.mce_irl(env, opt_tup, rmodel, optimal_om, print_interval=100, max_iter=1000)

In [ ]:
print('Optimal state visitation frequencies for each grid cell:')
print(optimal_om.reshape((random_gridworld.height, random_gridworld.width)))
print('\nRecovered state visitation frequencies for each grid cell:')
print(visitations.reshape((random_gridworld.height, random_gridworld.width)))

## More complex example showing how to use an EnvFeedbackModel to recover both a reward function + sub-rationality model

This code actually does use the new API to show how to use the 'blind IRL' feedback model (& its associated expert, which doesn't support observation blinding yet).

In [ ]:
irl_feedback_model = fbl_blind_irl.BlindIRLFeedbackModel(env)
rmodel = r_models.LinearRewardModel(env.obs_dim)
rng = jrandom.PRNGKey(42)
rng, irl_bias_params = irl_feedback_model.init_bias_params(rng)
irl_expert = experts.MEDemonstratorExpert(env, np.random.randint((1 << 31) - 1))
# we'll do IRL based on 10 trajectories
irl_dataset = irl_expert.interact(10)

In [ ]:
print('Log likelihood (IRL):', float(irl_feedback_model.log_likelihood(irl_dataset, rmodel, irl_bias_params)))
print('Gradient w.r.t. reward params (IRL):\n', np.asarray(irl_feedback_model.log_likelihood_grad_rew(irl_dataset, rmodel, irl_bias_params)))
print('Gradient w.r.t. bias params (IRL):\n', irl_feedback_model.log_likelihood_grad_bias(irl_dataset, rmodel, irl_bias_params))

## Another example with the paired comparison learner

In [ ]:
pc_feedback_model = fbl_paired_comp.PairedCompFeedbackModel(env)
rng, pc_bias_params = pc_feedback_model.init_bias_params(rng)
pc_expert = experts.PairedComparisonExpert(env, boltz_temp=1.0, seed=42)

# generate some random trajectories & compare a random subset of them
def generate_comparison_dataset(pc_ntraj):
    pc_trajectories = mce_irl.mce_irl_sample(env, pc_ntraj, R=np.ones((env.n_states, )))
    to_compare_first = np.arange(len(pc_trajectories['states']))
    comparisons = []
    for first_idx in range(pc_ntraj):
        second_idx = np.random.randint(pc_ntraj - 1)
        if second_idx >= first_idx:
            second_idx += 1
        traj1_is_better = pc_expert.interact(
            dict(states=pc_trajectories['states'][first_idx]),
            dict(states=pc_trajectories['states'][second_idx]))
        if traj1_is_better:
            # the better trajectory comes before the worse one
            comparisons.append((first_idx, second_idx))
        else:
            comparisons.append((second_idx, first_idx))
    return {
        'trajectories': pc_trajectories,
        'comparisons': np.asarray(comparisons),
    }

comparison_dataset = generate_comparison_dataset(10)

In [ ]:
print('Log likelihood (PC):', float(pc_feedback_model.log_likelihood(comparison_dataset, rmodel, pc_bias_params)))
print('Gradient w.r.t. reward params (PC):\n', np.asarray(pc_feedback_model.log_likelihood_grad_rew(comparison_dataset, rmodel, pc_bias_params)))
print('Gradient w.r.t. bias params (PC):\n', pc_feedback_model.log_likelihood_grad_bias(comparison_dataset, rmodel, pc_bias_params))

## Putting it all together

Now we can create a combined demo in which we use both MCE IRL and paired comparisons to learn a reward model and set of modality-specific bias models.

**TODO:** things I want to demo:

1. Joint training with a shared reward model, but separate bias models, all in the same environment.
2. **(Stretch goal)** Joint training with separate reward models across different environments, but using the same reward model. Again use several modalities per environment.

In [ ]:
# TODO(sam): try finding a step size with backtracking line search, rather than using a fixed step
def joint_training_step(env, feedback_models, all_datasets, all_bias_params, rmodel, rprior, step=1e-3):
    assert len(feedback_models) == len(all_datasets)
    assert len(feedback_models) == len(all_bias_params)
    all_bias_params = list(all_bias_params)

    # first do a joint reward model update w/ plain gradient descent
    rmodel_grad = np.zeros_like(rmodel.get_params())
    for model, dataset, bias_params in zip(feedback_models, all_datasets, all_bias_params):
        rmodel_grad += model.log_likelihood_grad_rew(dataset, rmodel, bias_params)
    rmodel_grad += rprior.log_prior_grad(rmodel.get_params())
    rmodel.set_params(rmodel.get_params() + step * rmodel_grad)

    # now do individual bias model updates
    for model_idx, (model, dataset, bias_params) in enumerate(zip(feedback_models, all_datasets, all_bias_params)):
        # Projected gradient descent: we take a gradient descent step, then
        # project back to the support of the prior.
        bias_param_grad = model.log_likelihood_grad_bias(dataset, rmodel, bias_params)
        bias_prior_grad = model.bias_prior.log_prior_grad(bias_params)
        bias_grad = bias_param_grad + bias_prior_grad
        unproj_step = all_bias_params[model_idx] + step * bias_grad
        proj_step = model.bias_prior.project_to_support(bias_grad)
        all_bias_params[model_idx] = proj_step

    return rmodel, all_bias_params